In [ ]:
import sys, os
sys.path.insert(0, '..')
# os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
import numpy as np
import pandas as pd
import torch
from models.weighted_node2vec import WeightedNode2Vec
from dataset import triplet_dataset, pokec_data
from utils.config import DEVICE
from tqdm import tqdm, trange
import gc
from utils.link_prediction import GCNLinkPrediction
import residual2vec as rv
import warnings
import faiss
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
gc.enable()

In [ ]:
window_length = 5
num_walks = 10
dim = 128
walk_length = 80
NUM_WORKERS = 4

In [ ]:
embs = {}

In [ ]:
embs['gcn_crosswalk'] = np.load("../data/pokec_crosswalk_gcn_embs.npy")

In [ ]:
emb = embs['gcn_crosswalk']

# distributions of positive and negative edgeswith dot product

In [ ]:
a, p, n = emb[:, :128].copy(), emb[:, 128: 128 * 2], emb[:, 128 * 2:]

In [ ]:
a.shape, p.shape, n.shape

In [ ]:
df = pd.DataFrame({
    "positive_dot": np.array([np.dot(a[i], p[i]) for i in trange(a.shape[0])]),
    "negative_dot": np.array([np.dot(a[i], n[i]) for i in trange(a.shape[0])])
})

In [ ]:
fig, ax = plt.subplots(1,1)
df.positive_dot.plot(kind='density')
df.negative_dot.plot(kind='density')
fig.show()

In [ ]:
df.positive_dot.describe()

In [ ]:
df.negative_dot.describe()

In [ ]:
del df, p, n
gc.collect()

In [ ]:
# create edge_list
k, batch_size = 5, 2000
from utils.graph_utils import get_edges_faiss
edges = get_edges_faiss(a, k, batch_size,)
edges